In [1]:
import h5py
import numpy as np
import pandas as pd
from pandas_profiling import ProfileReport
from tqdm import tqdm
from statistics import mean, stdev
import torch

In [2]:
mol_prop = pd.read_csv("/p/project/hai_drug_qm/Dataset/aeneas/DB/molprop.csv", sep=';')
mol_prop_head = ["pdbid",  "total_charge", "Electron_Affinity", "Electronegativity", "Hardness", "Ionization_Potential", "Koopman", "MW", "Polarisability"]
mol_prop.columns = ["pdbid",  "total_charge", "Electron_Affinity", "Electronegativity", "Hardness", "Ionization_Potential", "Koopman", "MW", "Polarisability"]


In [4]:
df_ele_aff

,pdbid,total_charge,Electron_Affinity,Electronegativity,Hardness,Ionization_Potential,Koopman,MW,Polarisability
0,6apu,-1.0,11.2920,11.6853,0.3933,12.0786,10.8581,354.28213,224.110
1,5zwf,0.0,6.1073,8.1155,2.0082,10.1237,8.8422,414.62160,311.710
2,5gsa,0.0,6.9428,8.9788,2.0360,11.0148,9.5481,369.39814,246.600
3,2fqy,0.0,2.5184,3.4917,0.9733,4.4651,2.2196,267.24172,161.560
4,3bh3,0.0,5.3227,8.3266,3.0039,11.3305,3.7169,86.13249,64.466
...,...,...,...,...,...,...,...,...,...
19451,6uyx,-2.0,6.6916,8.3465,1.6549,10.0014,9.5727,1091.15452,707.420
19452,6uyy,-5.0,6.3566,8.0395,1.6829,9.7224,9.4791,1348.33185,854.160
19453,6uyz,-1.0,6.1223,8.1423,2.0200,10.1623,9.6928,941.10452,621.000
19454,6v1c,0.0,4.4141,7.8524,3.4383,11.2907,8.4555,383.34898,216.050


In [3]:
from scipy import stats
df_ele_aff = mol_prop[(np.abs(stats.zscore(mol_prop["Electron_Affinity"])) < 20)]
df_hardness = df_ele_aff[(np.abs(stats.zscore(df_ele_aff["Hardness"])) < 20)]
df_elec_neg = df_hardness[(np.abs(stats.zscore(df_hardness["Electronegativity"])) < 20)]
df_ion = df_elec_neg[(np.abs(stats.zscore(df_elec_neg["Ionization_Potential"])) < 20)]
df_ion

,pdbid,total_charge,Electron_Affinity,Electronegativity,Hardness,Ionization_Potential,Koopman,MW,Polarisability
0,6apu,-1.0,11.2920,11.6853,0.3933,12.0786,10.8581,354.28213,224.110
1,5zwf,0.0,6.1073,8.1155,2.0082,10.1237,8.8422,414.62160,311.710
2,5gsa,0.0,6.9428,8.9788,2.0360,11.0148,9.5481,369.39814,246.600
3,2fqy,0.0,2.5184,3.4917,0.9733,4.4651,2.2196,267.24172,161.560
4,3bh3,0.0,5.3227,8.3266,3.0039,11.3305,3.7169,86.13249,64.466
...,...,...,...,...,...,...,...,...,...
19451,6uyx,-2.0,6.6916,8.3465,1.6549,10.0014,9.5727,1091.15452,707.420
19452,6uyy,-5.0,6.3566,8.0395,1.6829,9.7224,9.4791,1348.33185,854.160
19453,6uyz,-1.0,6.1223,8.1423,2.0200,10.1623,9.6928,941.10452,621.000
19454,6v1c,0.0,4.4141,7.8524,3.4383,11.2907,8.4555,383.34898,216.050


In [ ]:
mol_prop.merge(df_ion, how = 'outer' ,indicator=True).loc[lambda x : x['_merge']=='left_only']["pdbid"].to_list()

In [5]:
with h5py.File("/p/project/hai_drug_qm/atom3d/examples/smp_mol/data/qm/aeneas/h5/qm.h5", "r") as f:
    X = [key for key in f.keys()]

In [6]:
with open("/p/project/hai_drug_qm/atom3d/examples/smp_mol/data/qm/aeneas/mol_split/whole_data.txt", "w") as f:
    f.write('\n'.join(list(set(df_ion["pdbid"].to_list()) & set(X))))

In [31]:
with open("/p/project/hai_drug_qm/atom3d/examples/smp_mol/data/qm/aeneas/mol_split/test-Copy1.txt", "r") as g:
    mols = g.read().splitlines()
    
list(set(df_ion["pdbid"].to_list()) & set(mols))

['5g46',
 '2qnq',
 '1w70',
 '4a4o',
 '3blu',
 '2ya6',
 '4yef',
 '4otf',
 '4ewn',
 '3shj',
 '6mxd',
 '4x6n',
 '4u58',
 '4uvu',
 '5yso',
 '5hk9',
 '1z9h',
 '1oeb',
 '4dtt',
 '4mc9',
 '6gl9',
 '1m1b',
 '4hco',
 '1jvp',
 '5emk',
 '5l4m',
 '6g84',
 '3k5x',
 '3dgo',
 '6boe',
 '2rqu',
 '2r6y',
 '6euw',
 '1yy4',
 '6gnp',
 '2jh6',
 '3twu',
 '3hwn',
 '1w2h',
 '2y7i',
 '2std',
 '2qky',
 '4l2g',
 '3dbu',
 '5itd',
 '5zhj',
 '3l5f',
 '3i7c',
 '3bvb',
 '5gty',
 '6aji',
 '1swg',
 '4lxd',
 '3a73',
 '4ysi',
 '1l8g',
 '6qra',
 '3uvw',
 '4i7m',
 '4aia',
 '6o5a',
 '5m3a',
 '6nvi',
 '6f1x',
 '6pnn',
 '1pw6',
 '2a0c',
 '4kp6',
 '4c71',
 '4elb',
 '3kai',
 '4nku',
 '3f3c',
 '5ezg',
 '4lqg',
 '6rr0',
 '2zb1',
 '6ftp',
 '6bw4',
 '2flh',
 '4ks3',
 '2viy',
 '1g45',
 '4whl',
 '4z93',
 '5zh4',
 '2loz',
 '4oma',
 '4p0w',
 '2pog',
 '5dpx',
 '5qax',
 '4i74',
 '6bl1',
 '6m7i',
 '5kcx',
 '8cpa',
 '3lxl',
 '3cke',
 '1yds',
 '3uec',
 '4ch2',
 '5mts',
 '3kab',
 '3lp4',
 '4qfp',
 '966c',
 '5ads',
 '3w2o',
 '6r9s',
 '4ef6',
 

In [32]:
with open("/p/home/jusers/benassou1/juwels/hai_drug_qm/atom3d/examples/smp_mol/data/qm/aeneas/mol_split/test.txt", "w") as f:
    f.write("\n".join(list(set(df_ion["pdbid"].to_list()) & set(mols))))

In [9]:
path = "/p/project/hai_drug_qm/atom3d/examples/smp_mol/data/qm/aeneas/h5/qm.h5"
f =  h5py.File(path, 'r') 
with open("/p/project/hai_drug_qm/atom3d/examples/smp_mol/data/qm/aeneas/mol_split/train_norm.txt", "r") as g :
    ids_train = g.read().splitlines()
with open("/p/project/hai_drug_qm/atom3d/examples/smp_mol/data/qm/aeneas/mol_split/val_norm.txt", "r") as g :
    ids_val = g.read().splitlines()
with open("/p/project/hai_drug_qm/atom3d/examples/smp_mol/data/qm/aeneas/mol_split/test_norm.txt", "r") as g :
    ids_test = g.read().splitlines()
idss = {"train":ids_train, "val":ids_val, "test":ids_test}

targets = [label.decode("utf-8") for label in f["4xkc"]["mol_properties/column_names"][:]]
targets = targets[1:5]

with h5py.File('norm_qm2.hdf5', 'w') as ff: 
    
    for ids in idss:
        
        count = 0 

        subgroup = ff.create_group(ids)
        for i, label in enumerate(targets):
            if ids == "train":
                print(label)
                rslt = []
                count += 1
                for index, n in tqdm(enumerate(idss[ids])):
                    pitem = f[ids_train[index]]

                    rslt.append(torch.tensor(pitem["mol_properties/data"])[count].item())
                rslt = np.array(rslt)
                # std = np.array(stdev(rslt))

                mean_std = np.array(rslt.mean(), rslt.std())
                subsubgroup = subgroup.create_group(label)

                subsubgroup.create_dataset("mean", data=rslt.mean())
                subsubgroup.create_dataset("std", data=rslt.std())
            # subgroup.create_dataset("std", data=rslt.std())n 
            else : 
                subsubgroup = subgroup.create_group(label)

                subsubgroup.create_dataset("mean", data=0)
                subsubgroup.create_dataset("std", data=1)
            

Electron_Affinity


15504it [00:05, 3022.13it/s]


Electronegativity


15504it [00:05, 3069.00it/s]


Hardness


15504it [00:05, 3078.81it/s]


Ionization_Potential


15504it [00:05, 3039.09it/s]


In [20]:
g = h5py.File("p/home/jusers/benassou1/juwels/hai_drug_qm/atom3d/examples/smp_mol/data/qm/aeneas/h5/qm.h5", "r") 

FileNotFoundError: [Errno 2] Unable to open file (unable to open file: name = 'p/home/jusers/benassou1/juwels/hai_drug_qm/atom3d/examples/smp_mol/data/qm/aeneas/h5/qm.h5', errno = 2, error message = 'No such file or directory', flags = 0, o_flags = 0)

In [15]:
g["train"]["Hardness"]["std"][()]

0.7240225022392128

In [17]:
g["train"]["Electron_Affinity"]["mean"][()]

6.509927662250454

In [20]:
from scipy import stats
profile = ProfileReport(pd.DataFrame(df_ion[["Electron_Affinity", "Electronegativity",  "Hardness", "Ionization_Potential"]]), title="Pandas Profiling Report")
profile.to_file(output_file='qm_report.html')

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]

In [7]:
with h5py.File('/p/project/hai_drug_qm/atom3d/examples/lba_md/data/MD_dataset_mapped.hdf5', 'r') as f: 
    
    print(f["11gs"].keys())
    print(f["11gs"]["atoms_number"][:])
    x = f["11gs"]["atoms_number"][:]
    # print(f["11gs"]["atoms_residue"])
    # print(f["11gs"]["atoms_type"])
    # print(f["11gs"]["frames_bSASA"])
    # print(f["11gs"]["trajectory_coordinates"])
    # print(f["11gs"]["molecules_begin_atom_index"])
    #  X = [key for key in f.keys()]
    

<KeysViewHDF5 ['atoms_number', 'atoms_residue', 'atoms_type', 'frames_bSASA', 'frames_distance', 'frames_interaction_energy', 'frames_rmsd_ligand', 'molecules_begin_atom_index', 'trajectory_coordinates']>
[7 1 1 ... 1 1 1]


In [9]:
profile = ProfileReport(pd.DataFrame(x), title="Pandas Profiling Report")
profile.to_widgets()

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render widgets:   0%|          | 0/1 [00:00<?, ?it/s]

In [78]:
stats = pd.DataFrame()

with h5py.File('/p/home/jusers/benassou1/juwels/hai_drug_qm/atom3d/examples/smp_mol/data/qm/aeneas/h5/qm.hdf5', 'r') as f: 

    X = [key for key in f.keys()]
    
    molstat = []
    for x in X:
        vals = f[x]["mol_properties/data"][1:6] 
        tmp = [x]
        tmp.extend(list(vals))
        molstat.append(tmp)
       
       
       
    

In [79]:
stats = stats.append(molstat)
stats.columns = ["pdbid", "Electron_Affinity", "Electronegativity", "Hardness", "Ionization_Potential", "Koopman"]

In [80]:
stats.loc[stats["Electron_Affinity"] < -100,"Electron_Affinity" ] = -100
stats.loc[stats["Electronegativity"] < -200,"Electronegativity" ] = -200
stats.loc[stats["Hardness"] > 100,"Hardness" ] = 100
stats.loc[stats["Koopman"] > 10000,"Koopman" ] = 10000
stats.loc[stats["Koopman"] < -10000,"Koopman" ] = -10000

In [81]:
stats.describe()

,Electron_Affinity,Electronegativity,Hardness,Ionization_Potential,Koopman
count,19417.000000,19417.000000,19417.000000,19417.000000,19417.000000
mean,6.374203,8.384419,2.160523,10.617773,7.338508
std,3.408874,5.533381,2.958306,0.530122,178.819199
min,-100.000000,-200.000000,-1.027100,5.620800,-10000.000000
25%,6.116100,8.238900,1.711600,10.281400,8.833600
50%,6.630000,8.571500,1.955500,10.593900,9.291400
75%,7.132400,8.935900,2.258900,10.925400,9.674700
max,11.756600,12.035700,100.000000,14.599000,10000.000000


Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render widgets:   0%|          | 0/1 [00:00<?, ?it/s]

In [3]:
import contextlib
import gzip
import importlib
import json
import io
import logging
import msgpack
import os
from pathlib import Path
import pickle as pkl
import tqdm
import urllib.request
import subprocess
import pickle 

#import Bio.PDB
# import lmdb
import h5py
import numpy as np
import pandas as pd
from torch.utils.data import Dataset, IterableDataset
import torch
import torch_geometric.transforms as T

try:
    import torch_geometric.data as ptg
except:
    print('torch geometric not found, GNN examples will not work until it is.')
    ptg = None

import atom3d.util.rosetta as ar
import atom3d.util.file as fi
import atom3d.util.formats as fo
from atom3d.util.transforms import mol_graph_transform, prot_graph_transform
import atom3d.util.graph as gr

from torch_geometric.data import Data

logger = logging.getLogger(__name__)

class MolH5Dataset(Dataset):
    """
    Creates a dataset from an h5 file. Adapted from `TAPE <https://github.com/songlab-cal/tape/blob/master/tape/datasets.py>`_.

    :param data_file: path to h5 file containing dataset
    :type data_file: Union[str, Path]
    :param transform: Transformation function to apply to each item.
    :type transform: Function, optional

    """

    def __init__(self, data_file, idx_file, target, transform=None, post_transform=None):
        """constructor

        """
        if type(data_file) is list:
            if len(data_file) != 1:
                raise RuntimeError("Need exactly one filepath for h5")
            data_file = data_file[0]

        self.data_file = Path(data_file).absolute()
        print(self.data_file)
        if not self.data_file.exists():
            raise FileNotFoundError(self.data_file)

        with open(idx_file, 'r') as f:  # this file will keep all complexe
            self.ids = f.read().splitlines()

        self.f = h5py.File(self.data_file, 'r') 
        self.target = target
        self.target_dict = {
            "Electron_Affinity" : 1,
            "Electronegativity" : 2,
            "Hardness" : 3,
            "Ionization_Potential" : 4,
            "Koopman" : 5
        }

        self._transform = transform
        self._post_transform = post_transform

    def __len__(self) -> int:
        return len(self.ids)

    def get(self, id: str):
        return self.data[id][:]

    def __getitem__(self, index: int):
        if not 0 <= index < len(self.ids):
            raise IndexError(index)

        pitem = self.f[self.ids[index]]
        
        column_names = ["x", "y", "z", "element"]

        atoms = pd.DataFrame(columns = column_names)
        prop = pitem["atom_properties/data"][:]
        atoms["x"] = prop[:,1]
        
        atoms["y"] = prop[:,2]
        
        atoms["z"] = prop[:,3]
        
        atoms["element"] = np.array([element.decode('utf-8') for element in pitem["atom_properties/atom_names"][:]])
        
        bonds = pitem["atom_properties/bonds"][:]
        
        scores = torch.tensor(pitem["mol_properties/data"])[self.target_dict[self.target]]

        item = {"atoms" : atoms,
        "labels": scores,
        "bonds": bonds, 
        "id": self.ids[index]}
 
        if self._transform:
            item = self._transform(item)

        if self._post_transform:
            item = self._post_transform(item)
       
        return item



In [8]:
from cnn3d.data import CNN3D_TransformSMP

h5_file = "/p/project/hai_drug_qm/atom3d/examples/smp_mol/data/qm/aeneas/h5/qm.hdf5"

train_idx = "/p/project/hai_drug_qm/atom3d/examples/smp_mol/data/qm/aeneas/mol_split/train.txt"
val_idx = "/p/project/hai_drug_qm/atom3d/examples/smp_mol/data/qm/aeneas/mol_split/val.txt"
test_idx = "/p/project/hai_drug_qm/atom3d/examples/smp_mol/data/qm/aeneas/mol_split/test.txt"


post_transform = T.Compose([
        # T.RandomJitter(0.5),
        T.RandomFlip(0,0.5),
        T.RandomScale((0.5, 0.5))
    ])

train_dataset = MolH5Dataset(h5_file, train_idx, target="Hardness", transform=CNN3D_TransformSMP("Hardness", random_seed=int(np.random.randint(1, 10e6))), post_transform=post_transform)
val_dataset = MolH5Dataset(h5_file, val_idx, target="Hardness", transform=CNN3D_TransformSMP("Hardness", random_seed=int(np.random.randint(1, 10e6))), post_transform=post_transform)
test_dataset = MolH5Dataset(h5_file, test_idx, target="Hardness", transform=CNN3D_TransformSMP("Hardness", random_seed=int(np.random.randint(1, 10e6))), post_transform=post_transform)


/p/project/hai_drug_qm/atom3d/examples/smp_mol/data/qm/aeneas/h5/qm.hdf5
/p/project/hai_drug_qm/atom3d/examples/smp_mol/data/qm/aeneas/h5/qm.hdf5
/p/project/hai_drug_qm/atom3d/examples/smp_mol/data/qm/aeneas/h5/qm.hdf5


In [9]:
from torch.utils.data import DataLoader

train_loader = DataLoader(train_dataset, 128, shuffle=True)
val_loader = DataLoader(val_dataset, 128, shuffle=False)
test_loader = DataLoader(test_dataset, 128, shuffle=False)


for data in train_loader:
    in_channels, spatial_size = data['feature'].size()[1:3]
    print('num channels: {:}, spatial size: {:}'.format(in_channels, spatial_size))
    break

AttributeError: 'dict' object has no attribute 'pos'

In [11]:

h5_file = "/dev/shm/qm.hdf5"

train_idx = "/dev/shm/train.txt"
val_idx = "/dev/shm/val.txt"
test_idx = "/dev/shm/test.txt"


# post_transform = T.Compose([
#         # T.RandomJitter(0.5),
#         T.RandomFlip(args.axis,args.flip),
#         T.RandomScale((args.scale, args.scale))
#     ])

train_dataset = MolH5Dataset(h5_file, train_idx, target="Hardness", transform=GNNTransformSMP("Hardness"), post_transform=post_transform)
val_dataset = MolH5Dataset(h5_file, val_idx, target="Hardness", transform=GNNTransformSMP("Hardness"), post_transform=post_transform)
test_dataset = MolH5Dataset(h5_file, test_idx, target="Hardness", transform=GNNTransformSMP("Hardness"), post_transform=post_transform)

train_dataloader = DataLoader(train_dataset, args.batch_size, shuffle=True, num_workers=8)
val_dataloader = DataLoader(val_dataset, args.batch_size, shuffle=False, num_workers=8)
test_dataloader = DataLoader(test_dataset, args.batch_size, shuffle=False, num_workers=8)

print("data loaded")
for data in train_dataloader:     
    # print(data)   
    num_features = data.num_features
    break

NameError: name 'args' is not defined